In [93]:
import requests
from bs4 import BeautifulSoup
import os 
import pandas as pd 
from tqdm import tqdm

In [94]:
CARD_NAME_LINK = 'https://www.db.yugioh-card.com/yugiohdb/card_search.action?ope=2&cid='

In [122]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
from tqdm import tqdm

CARD_NAME_LINK = 'https://www.db.yugioh-card.com/yugiohdb/card_search.action?ope=2&cid='

###### FUNCTIONS ######

def crawl_card(url, locale):
    # Fetch the HTML content of the webpage
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        
        try:
            # Find card name
            card_name = get_name(soup, locale)
            if card_name == '':
                return '', '', '', ''
            
            # Crawl description
            card_description, card_pen = get_description(soup)
            
            # Crawl side content
            side_content = get_side_content(soup)
        except: 
            print(f"Error at {url}")
        
    else:
        return '', '', '', ''
        
    return card_name, card_description, card_pen, side_content


def get_name(soup_content, locale):
    # Find card name
    try:
        div_element = soup_content.find('div', {'id': 'cardname'})
        card_name = div_element.h1.text.strip()
        
        if locale == 'ja':
            card_name = card_name.split('\n\t\n\t\t\t')[1]
    except:
        card_name = ''
    return card_name


def get_description(soup_content):
    des_text = soup_content.find_all('div', {'class': 'item_box_text'})
    if len(des_text) == 1:
        pen_eff = ''
        mons_eff = des_text[0].text
    
    else:
        pen_eff = des_text[0].text.strip()
        mons_eff = des_text[1].text
    
    return mons_eff.split('\n\t\t\t\t\t\n\t\t\t\t\t')[1].strip(), pen_eff
    
    
def get_side_content(soup_content):
    try:
        spec = soup_content.find('p', {'class': 'species'})
        return spec.text.replace('\n', '').strip()
    except:
        return ''


def crawl_id(id):
    ## EN ##
    url_en = CARD_NAME_LINK + str(id) + '&request_locale=en'
    card_name_en, card_description_en, card_pen_en, side_content_en = crawl_card(url_en, 'en')
    
    ## JA ##
    url_ja = CARD_NAME_LINK + str(id) + '&request_locale=ja'
    card_name_ja, card_description_ja, card_pen_ja, side_content_ja = crawl_card(url_ja, 'ja')
    
    return card_name_en, side_content_en, card_description_en, card_pen_en, card_name_ja, side_content_ja, card_description_ja, card_pen_ja
    
    
def creat_card_dataset(first, last, save_path = 'data'):
    i = 0
    df = pd.DataFrame(columns=['id', 'name_en', 'side_content_en', 'description_en', 'pen_eff_en', 'name_ja', 'side_content_ja', 'description_ja', 'pen_eff_ja', 'available'])
    
    # Start crawling
    print(f"Start crawling from {first} to {last}")
    
    for id in tqdm(range(first, last)):
        card = crawl_id(id)
        
        # Add card to dataframe
        if (card[0] is not '') and (card[4] is not ''):
            available = True

            
        df.loc[0] = [id, card[0], card[1], card[2], card[3], card[4], card[5], card[6], card[7], available]
        
        i+= 1
        
    # Save dataframe
    df.to_csv(save_path + f"/card_dataset_{first}_{last}.csv", index=False)
        
    return 

In [131]:
crawl_card('https://www.db.yugioh-card.com/yugiohdb/card_search.action?ope=2&cid=19360', 'en')

('Mementoal Tecuhtlica - Combined Creation',
 'Cannot be Normal Summoned/Set. Must be Special Summoned (from your hand or GY) by shuffling 5 other "Memento" monsters with different names from your hand and/or GY into the Deck/Extra Deck. While you control no other monsters, this card can attack all monsters your opponent controls, once each. Once per turn, if your opponent activates a card or effect (except during the Damage Step): You can Special Summon 1 "Memento" monster from your hand or GY.',
 None,
 'Wyrm/Effect')

In [104]:
# Fetch the HTML content of the webpage
response = requests.get('https://www.db.yugioh-card.com/yugiohdb/card_search.action?ope=2&cid=11134')

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

In [109]:
soup


<!DOCTYPE html>

<html>
<head>
<meta content="Stargazer Magician,KONAMI,yugioh,Yu-Gi-Oh,TCG,card,database,card,list,Forbidden,Limited,Semi-limited,Cards," name="keywords"/>
<meta content="Please check this page to see the official Yu-Gi-Oh! TCG rules for {0}. You can also search for Decks containing this card. The Yu-Gi-Oh! TCG Card Database is an official Konami Site for the Yu-Gi-Oh! Trading Card Game." name="description"/>
<meta content="Stargazer Magician | Card Details | Yu-Gi-Oh! TRADING CARD GAME - CARD DATABASE" name="title"/>
<meta content="Stargazer Magician | Card Details | Yu-Gi-Oh! TRADING CARD GAME - CARD DATABASE" property="og:title"/>
<meta content="Please check this page to see the official Yu-Gi-Oh! TCG rules for {0}. You can also search for Decks containing this card. The Yu-Gi-Oh! TCG Card Database is an official Konami Site for the Yu-Gi-Oh! Trading Card Game." property="og:description"/>
<meta content="Stargazer Magician | Card Details | Yu-Gi-Oh! TRADING CARD GA

In [110]:
soup.find_all('div', {'class': 'item_box_text'})

[<div class="item_box_text">
 			
 							If a Pendulum Monster you control battles, your opponent cannot activate Spell Cards until the end of the Damage Step. Unless you have a "Magician" card or "Odd-Eyes" card in your other Pendulum Zone, this card's Pendulum Scale becomes 4.
 			
 					</div>,
 <div class="item_box_text">
 <div class="text_title">
 						Card Text
 					</div>
 					Once per turn, when exactly 1 other Pendulum Monster you control (and no other cards) is returned to your hand by an opponent's card effect (except during the Damage Step): You can Special Summon 1 monster from your hand with the same name as that returned to the hand.
 				</div>]

In [91]:
des_list = soup.find_all('div', {'class': 'item_box_text'})
des_list

[<div class="item_box_text">
 <div class="text_title">
 						Card Text
 					</div>
 					Cannot be Normal Summoned/Set. Must be Special Summoned (from your hand or GY) by shuffling 5 other "Memento" monsters with different names from your hand and/or GY into the Deck/Extra Deck. While you control no other monsters, this card can attack all monsters your opponent controls, once each. Once per turn, if your opponent activates a card or effect (except during the Damage Step): You can Special Summon 1 "Memento" monster from your hand or GY.
 				</div>]

In [92]:
len(des_list)

1

In [ ]:
pen_eff = des_list[0].text.strip()
pen_eff

In [88]:
mons_eff = des_list[1].text.strip()
mons_eff

"Card Text\n\t\t\t\t\t\n\t\t\t\t\tOnce per turn, when exactly 1 other Pendulum Monster you control (and no other cards) is returned to your hand by an opponent's card effect (except during the Damage Step): You can Special Summon 1 monster from your hand with the same name as that returned to the hand."